<head>

<script type="text/javascript"
  src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML">
</script>

</head>
<style>


@font-face {
    font-family: "Computer Modern";
    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');
}
#notebook_panel { /* main background */
    background: #888;
    color: #f6f6f6;
}
#notebook li { /* More space between bullet points */
margin-top:0.8em;
}
div.text_cell_render{
    font-family: 'Arvo' sans-serif;
    line-height: 130%;
    font-size: 135%;
    width:1000px;
    margin-left:auto;
    margin-right:auto;
}

</style>


<center>

<p class="gap05"<p>
<h1>Intro to SQL:</h1>
<h2>Aggegation, Joins, Subqueries, Set Operations, ...</h2>

<p class="gap05"<p>
<h3>Darrell Aucoin</h3>

<h3>Stats Club</h3>

<p class="gap2"<p>
![alt text](Images/exploits_of_a_mom.png)
</center>

<style type="text/css">
.input_prompt, .input_area, .output_prompt {
    display:none !important;
}
</style>
Source for picture: XKCD (Randall Munroe) http://xkcd.com/327/

## Content (This Presentation)

1. Filtering rows (WHERE)
2. Aggregation (GROUP BY)
3. Joining tables
4. Subqueries
5. Set Operations
6. WITH Clause (Increases Readiablity)
7. CASE expressions
8. ORDER BY Clause
9. LIMIT Clause  

# How to Follow Along

#### Get the material  

1. Go to https://github.com/NormallySane/IntroSQL
2. Download [zip file](https://github.com/NormallySane/IntroSQL/archive/master.zip)
    ![alt text](Images/IntroSQLGithub.png)
3. Unzip downloaded file in your preferred directory

#### Using SQLite Browser

1. Open SQLite browser
2. Open `stats_club.db` database file in the downloaded directory
3. Click on 'Execute SQL' tab
    1. Open SQL file `IntroSQL_Presentation2.sql` file in the downloaded directory
    2. Follow along with file, executing statement as topics dictate
4. Content of the talk is on https://github.com/NormallySane/IntroSQL/wiki (open in your favorite browser)


- SQLite browser is a great tool for learning SQLite and SQL in general

## Selection (WHERE Clause)

__WHERE__ clauses filters the result set, removing rows where the condition returns either FALSE or NULL.

```
SELECT col1, col2, ...
FROM table_name
WHERE condition;
```

__Example:__ What Stats Club events are social?

```
SELECT name, type, start_time, end_time, location 
FROM event 
WHERE type = 'social';
```

In [1]:
import pandas as pd
import sqlite3 as sql
con = sql.connect('stats_club.db')

In [2]:
pd.read_sql_query(
"SELECT name, type, start_time, end_time, location \
FROM event \
WHERE type = 'social'", con)

,name,type,start_time,end_time,location
0,BOT,social,2015-01-28 19:00:00,2015-01-28 22:00:00,C & D
1,EOT,social,None,None,None


### Prediate Operators

| Operator | Description | Example |
|----------|-------------|---------|
| = | Equal to | WHERE gender = 'M' |
| <>, != | Not equal to | WHERE gender <> 'M' |
| > | Greater than | WHERE num > 5 |
| < | Less than | WHERE num < 5 |
| >= | Greater than or equal to | WHERE num >= 5 |
| <= | Less than or equal to | WHERE num <= 5 |
| IS NULL | Value is NULL | WHERE num IS NULL |
| IS NOT NULL | Value is not NULL | WHERE num IS NOT NULL |
| BETWEEN | Between a range | WHERE num BETWEEN 3 AND 5 |
| IN | In a list of values | WHERE num IN (3, 5, 8) |
| LIKE | Pattern Search | WHERE str LIKE 'F%' |
| EXISTS | Subquery have any rows? | WHERE EXISTS (subquery) |

#### LIKE Predicate

LIKE predicates 

'\_' means a character of any type  
'%' means between 0 or more characters of any type

__Example:__ What Stats Club members has a name begining with F?

```
SELECT * 
FROM member 
WHERE name LIKE 'F%';
```

In [3]:
pd.read_sql_query(
"SELECT * \
FROM member \
WHERE name LIKE 'F%'", con)

,quest_id,name,email,faculty,major,paid,card
0,fred,Fred E. Finch,fred@uwaterloo.ca,Math,Pure Math,Y,Y
1,frances,Frances A. Miller,frances@uwaterloo.ca,Math,Stats,Y,Y


#### Operator Modifiers

##### ALL, ANY or SOME Operator Modifiers

The operators =, <>, !=, >, <, >=, <= can be used with a list of values and the operators `ALL` or `ANY / SOME`.  

- SQLite does __NOT__ have `ALL` or `ANY / SOME` implemented

__ANY, SOME__ Operator returns true, if operator is true for __any__ value in the set.  

$$E_{1}\vee E_{2}\vee E_{3}\vee\cdots\vee E_{n}$$

__ALL__ Operator returns true, if operator is true for __all__ values in the set.  

$$E_{1}\wedge E_{2}\wedge E_{3}\wedge\cdots\wedge E_{n}$$

```
SELECT *
FROM table_name
WHERE column_name < ALL (subquery_returning_one_column);
```

__Example:__ What are the expenses for non-social events? 
- (`ALL`, `ANY`, and `SOME` are not implmented in SQLite)

```
SELECT * 
FROM expenses 
WHERE event != ALL (SELECT name 
                    FROM event
                    WHERE type = 'social');
```

In [4]:
pd.read_sql_query(
"SELECT * \
FROM expenses \
WHERE event NOT IN (SELECT name \
                    FROM event \
                    WHERE type = 'social')", con)

,event,expense,price
0,Intro to SQL,pizza,87.43
1,Intro to SQL,pop,15.34
2,Intro to Hadoop,coffee,23.12
3,Intro to Hadoop,water,10.23
4,Intro to Hadoop,donuts,53.23
5,Intro to Hadoop,cookies,10.23
6,Intro to SQL: Basic Queries,cookies,10.23
7,Intro to SQL: Basic Queries,donuts,20.34
8,Intro to SQL: Basic Queries,pop,21.54
9,Intro to SQL: Basic Queries,water,10.52


#### AND, OR Operators

A group of filter conditions can be linked together with AND or OR operators. 

```
SELECT col1, col2, ...
FROM table_name
WHERE (condition1 AND condition2 ) OR condition3;
```

__Example:__ What Stats Club members has a name with it's second letter as A or ends in B?

```
SELECT name
FROM member 
WHERE name LIKE '_a%' OR  name LIKE '%b';
```

In [5]:
pd.read_sql_query(
"SELECT name \
FROM member \
WHERE name LIKE '_a%' OR  name LIKE '%b'", con)

,name
0,Darrell Aucoin
1,James M. Eddings
2,James A. Foxt
3,Daniel J. Moore
4,Nancy P. Jackson
5,Ralph L. Waldrop
6,Tameika M. McMaster
7,Janelle T. Smith
8,Ruben Lamb
9,Patrick Robertson


### Quiz: Filtering (WHERE Clause)

__Q:__ What events for Stats Club are introductory talks?
- Introductory talk names start with 'Intro'

```
SELECT name
FROM event 
WHERE name LIKE 'Intro%';
```

In [6]:
pd.read_sql_query(
"SELECT name \
FROM event \
WHERE name LIKE 'Intro%'", con)

,name
0,Intro to Hadoop
1,Intro to SQL
2,Intro to SQL: Advanced Queries
3,Intro to SQL: Basic Queries


__Q:__ What Stats Club members have their first name starting with a letter __BETWEEN__ A and G?

```
SELECT name 
FROM member 
WHERE name BETWEEN 'A' AND 'G';
```

In [7]:
pd.read_sql_query(
"SELECT name \
FROM member \
WHERE name BETWEEN 'A' AND 'G'", con)

,name
0,Darrell Aucoin
1,Fred E. Finch
2,Billy L. Hunter
3,Austin K. Gilliard
4,Elaine S. Ott
5,Daniel J. Moore
6,Frances A. Miller
7,Deborah D. Helfrich
8,Bobbie D. Mathews
9,Arnold J. Fuller


## GROUP BY Clause

- GROUP BY clause groups the table by a column (or tuple of columns) and applies a function to each group
    - In the SELECT portion of the statement, you can only list the grouped column(s) and aggregate functions on them.  
<center>
![alt text](Images/Group By.png)
</center>

```
SELECT col1, col2, ..., aggregate_function
FROM table_name
GROUP BY col1, col2, ...;
```

### Aggregate Functions

Recall:
__Aggregate Functions:__ Takes in the columns of a table and aggregates over the entries. 
- If we use a GROUP BY clause, the aggregation will be over those groups

| Function | Return value |
|----------|--------------|
| AVG(column) | Average of non-null values |
| COUNT(column) | Count of non-null values |
| MAX(column) | Maximum of values |
| MIN(column) | Minimum of values |
| SUM(column) | Sum of values |
| GROUP_CONCAT(column) | Concatenation of column strings |

- There are more aggregate functions for other implementations of SQL  

__Example:__ What are the number of each type of event for Stats Club?

```
SELECT type, COUNT(*) AS num_events 
FROM event 
GROUP BY type;
```

In [2]:
pd.read_sql_query(
"SELECT type, COUNT(*) AS num_events \
FROM event \
GROUP BY type", con)

,type,num_events
0,educational,6
1,social,2


### HAVING Clause

HAVING clauses are very similar to WHERE clauses but can have aggregate function in their conditions.

You can have a WHERE and HAVING clause in the same statement.

__Example:__ How many Stats Club members are in each major where the major has at least 2 members?

```
SELECT faculty, major, COUNT(*) 
FROM member 
GROUP BY faculty, major 
HAVING COUNT(*) >= 2;
```

In [3]:
pd.read_sql_query(
"SELECT faculty, major, COUNT(*) \
FROM member \
GROUP BY faculty, major \
HAVING COUNT(*) >= 2", con)

,faculty,major,COUNT(*)
0,Math,Act Sci,10
1,Math,Applied Math,2
2,Math,C & O,2
3,Math,CS,3
4,Math,Pure Math,2
5,Math,Stats,14


### GROUP BY with ROLLUP / CUBE

The ROLLUP operator produces a result set where the aggregate function is applied to each level of the GROUP BY hierachy.
- The ROLLUP operator is __NOT__ implemented in SQLite
- Useful for making reports with totals and subtotals

```
SELECT col1, col2, ..., aggregate_function
FROM table_name
GROUP BY col1, col2, ... WITH ROLLUP;
```

__Example:__ What are the number of Stats Club members in each faculty and major, including subtotals?

```
SELECT faculty, major, COUNT(*) AS num_members
FROM member
GROUP BY faculty, major WITH ROLLUP;
```

In [4]:
pd.read_sql_query(
"SELECT faculty, major, num_members FROM (  \
SELECT '1' orderCol, faculty, major, COUNT(*) AS num_members FROM member GROUP BY faculty, major \
UNION \
SELECT '2' orderCol, faculty, NULL as major, COUNT(*) AS num_members FROM member \
GROUP BY faculty \
UNION \
SELECT '99' orderCol, NULL AS faculty, NULL AS major, COUNT(*) AS num_members FROM member) AS t1 \
ORDER BY CASE when orderCol='99' then 1 else 0 end, faculty, orderCol", con)

,faculty,major,num_members
0,Art,Econ,1
1,Art,None,1
2,Math,Act Sci,10
3,Math,Applied Math,2
4,Math,C & O,2
5,Math,CS,3
6,Math,Pure Math,2
7,Math,Stats,14
8,Math,None,33
9,None,None,34


Note that for Art and Math in faculty there is a row which has a NULL value. This is a total for those groups.  

There is also one row with NULL values for faculty and major, this is the grand total of all members.

### Quiz: Aggregation (GROUP BY Clause)

__Q:__ What is the attendance for each Stats Club event?

```
SELECT event, COUNT(*) AS attendance 
FROM attendance 
GROUP BY event;
```

In [5]:
pd.read_sql_query(
"SELECT event, COUNT(*) AS attendance \
FROM attendance \
GROUP BY event", con)

,event,attendance
0,BOT,12
1,EOT,18
2,Intro to Hadoop,17
3,Intro to SQL,22
4,Intro to SQL: Advanced Queries,23
5,Intro to SQL: Basic Queries,23
6,Prof Talk,20
7,Prof Talk 2,16


## Joins

At times, we need information from multiple tables, to do this we need to join tables together. We can do this several ways:

1. __CROSS JOIN:__ The cartesian product of rows from each table.
2. __INNER JOIN:__ Join two tables on a join-predicate, losing rows when evaluated false/null.
3. __OUTER JOIN:__ Retains each record for the table(s) even when it has no matching rows from the other table. The returning table has null values for missing records.

    1. __LEFT OUTER JOIN:__ Keep each record for first table but not the table it's joining with.

    2. __RIGHT OUTER JOIN:__ Keep each record for second table but not the table it's joining with.

    3. __FULL OUTER JOIN:__ Keep all record for all tables.

4. __NATURAL JOIN:__ Tables with the exact same column name and datatype are joined along that column.

### CROSS JOIN

__CROSS JOIN__ is the cartesian product of two tables
```
SELECT col1, col2, ...
FROM table1 CROSS JOIN table2;
```
- if table1 has n rows and table2 has m rows, then the result set has n * m rows  

We can also get a CROSS JOIN by listing the tables seperated by a ','
```
SELECT col1, col2, ...
FROM table1, table2;
```

__Example:__ Suppose we are creating a games tourtament between Stats Club members where every member play every other member once. How can we create such a table? 

```
SELECT m1.name AS Player_1, m2.name  AS Player_2 
FROM member AS m1 CROSS JOIN member AS m2;
```

In [6]:
pd.read_sql_query(
"SELECT m1.name AS Player_1, m2.name  AS Player_2 \
FROM member AS m1 CROSS JOIN member AS m2", con)

,Player_1,Player_2
0,Darrell Aucoin,Darrell Aucoin
1,Darrell Aucoin,Fred E. Finch
2,Darrell Aucoin,Ryan T. Luby
3,Darrell Aucoin,Billy L. Hunter
4,Darrell Aucoin,John J. Oquendo
5,Darrell Aucoin,Stephanie R. Matthews
6,Darrell Aucoin,Robert B. Williams
7,Darrell Aucoin,Austin K. Gilliard
8,Darrell Aucoin,James M. Eddings
9,Darrell Aucoin,Elaine S. Ott


However we have have players playing themselves, and rounds with  the same players in opposite roles. We just need to filter these out.
```
SELECT m1.name AS Player_1, m2.name  AS Player_2 
FROM member AS m1 CROSS JOIN member AS m2 
WHERE m1.name > m2.name;
```

In [7]:
pd.read_sql_query(
"SELECT m1.name AS Player_1, m2.name  AS Player_2 \
FROM member AS m1 CROSS JOIN member AS m2 \
WHERE m1.name > m2.name", con)

,Player_1,Player_2
0,Darrell Aucoin,Billy L. Hunter
1,Darrell Aucoin,Austin K. Gilliard
2,Darrell Aucoin,Daniel J. Moore
3,Darrell Aucoin,Bobbie D. Mathews
4,Darrell Aucoin,Arnold J. Fuller
5,Darrell Aucoin,Ann W. McLaughlin
6,Fred E. Finch,Darrell Aucoin
7,Fred E. Finch,Billy L. Hunter
8,Fred E. Finch,Austin K. Gilliard
9,Fred E. Finch,Elaine S. Ott


### INNER JOIN

__INNER JOIN__ Joins two tables where the join condition returns true. Discarded when returning false or NULL.
```
SELECT col1, col2, ...
FROM table1 INNER JOIN table2 ON condition;
```

#### ON Clause

The __ON__ clause specifies the join condition:

- The ON clause can use a multiple set of conditions connected by AND, OR

- USING(<join col>) can also be used if both tables have the same column name and type

- Some SQL implementations constructs the ON clause from the WHERE clause (DB2)
    - filtering by the WHERE clause gives the same result but in some implementations will product an intermediate cross product of tables (making the query slower)

__Example:__ Note that in the previous example with CROSS JOIN where we made a tournament player list using the WHERE clause, we can get the same table by using an INNER JOIN and Θ='>'.

```
SELECT m1.name AS Player_1, m2.name  AS Player_2 
FROM member AS m1 INNER JOIN member AS m2 ON m1.name > m2.name;
```

In [8]:
pd.read_sql_query(
"SELECT m1.name AS Player_1, m2.name  AS Player_2 \
FROM member AS m1 INNER JOIN member AS m2 ON m1.name > m2.name", con)

,Player_1,Player_2
0,Darrell Aucoin,Billy L. Hunter
1,Darrell Aucoin,Austin K. Gilliard
2,Darrell Aucoin,Daniel J. Moore
3,Darrell Aucoin,Bobbie D. Mathews
4,Darrell Aucoin,Arnold J. Fuller
5,Darrell Aucoin,Ann W. McLaughlin
6,Fred E. Finch,Darrell Aucoin
7,Fred E. Finch,Billy L. Hunter
8,Fred E. Finch,Austin K. Gilliard
9,Fred E. Finch,Elaine S. Ott


__Note:__ `CROSS JOIN` is rarely used in practice as `INNER JOIN` is more efficient and usually the same `WHERE` condition can be applied to the ON clause.

__Example:__ How many events does each member attend?

- Note that this query does not include members who attended no events
    - Query is ordered by events_attended to show a comparison with a latter query  

```
SELECT m.name, COUNT(a.event) AS events_attended 
FROM member AS m INNER JOIN attendance AS a ON m.quest_id = a.member
GROUP BY m.name
ORDER BY events_attended;
```

In [9]:
pd.read_sql_query(
"SELECT m.name, COUNT(a.event) AS events_attended \
FROM member AS m INNER JOIN attendance AS a ON m.quest_id = a.member \
GROUP BY m.name \
ORDER BY events_attended", con)

,name,events_attended
0,John J. Oquendo,1
1,Melissa R. Anderson,2
2,Melvin O. Martin,2
3,Mina W. Lawrence,2
4,Ann W. McLaughlin,3
5,Bobbie D. Mathews,3
6,James M. Eddings,3
7,Austin K. Gilliard,4
8,Janelle T. Smith,4
9,Jeff M. Wright,4


### OUTER JOIN

__OUTER JOIN__ A join that returns all rows for 1 or 2 tables, even when there is no corresponding value. In these cases, NULL values are entered for these corresponding rows.

There are 3 types of OUTER JOINs:

1. __LEFT OUTER JOIN__: An OUTER JOIN returning all rows of the table first mentioned.

2. __RIGHT OUTER JOIN__: An OUTER JOIN returning all rows of the table second mentioned.

3. __FULL OUTER JOIN__: An OUTER JOIN returning all rows of both tables.  


- Only LEFT OUTER JOIN is implemented in SQLite

__Example:__ What are the names of Stat Club members and how many events they attended?

```
SELECT m.name, COUNT(a.event) AS events_attended 
FROM member AS m LEFT OUTER JOIN attendance AS a ON m.quest_id = a.member
GROUP BY m.name
ORDER BY events_attended;
```

In [10]:
pd.read_sql_query(
"SELECT m.name, COUNT(a.event) AS events_attended \
FROM member AS m LEFT OUTER JOIN attendance AS a ON m.quest_id = a.member \
GROUP BY m.name \
ORDER BY events_attended", con)

,name,events_attended
0,Darrell Aucoin,0
1,James A. Foxt,0
2,John J. Oquendo,1
3,Melissa R. Anderson,2
4,Melvin O. Martin,2
5,Mina W. Lawrence,2
6,Ann W. McLaughlin,3
7,Bobbie D. Mathews,3
8,James M. Eddings,3
9,Austin K. Gilliard,4


### Natural Join

__NATURAL JOIN__ A join condition that lets the server decide on the join conditions based on the same column names and types across columns for the tables.

__Example:__ What are the position and duties of each Stats Club exec?

```
SELECT  e.name, e.position, ep.duties 
FROM exec AS e NATURAL JOIN exec_position AS ep;
```

In [11]:
pd.read_sql_query(
"SELECT  e.name, e.position, ep.duties \
FROM exec AS e NATURAL JOIN exec_position AS ep", con)

,name,position,duties
0,Darrell Aucoin,President,To be aware of MathSoc's Policies and Bylaws i...
1,Darrell Aucoin,President,To call and preside over general meetings.
2,Darrell Aucoin,President,To manage the executive team and the strategic...
3,Darrell Aucoin,President,"To post announcements of all club meetings, an..."
4,Judith B. Gibbons,Events,To assist the president and other vice-preside...
5,Judith B. Gibbons,Events,To chair the organization and promotion of lea...
6,Lamar Roy,Finance,To ensure membership fees are collected and ma...
7,Lamar Roy,Finance,To keep an up-to-date record of financial tran...
8,Lamar Roy,Finance,To prepare a summary of the financial records ...
9,Lamar Roy,Finance,To prepare the budget at the beginning of term.


### Quiz: Joining Tables

__Q:__ What are the email addresses and phone numbers of stats club execs who are in change or organizing at least one event?
<center>
![alt text](Images/StatClubTables2.png)
</center>

```
SELECT DISTINCT exec.name, exec.email, exec.phone 
FROM exec INNER JOIN event ON exec.questid = event.organizer;
```

In [12]:
pd.read_sql_query(
"SELECT DISTINCT exec.name, exec.email, exec.phone \
FROM exec INNER JOIN event ON exec.questid = event.organizer", con)

,name,email,phone
0,Judith B. Gibbons,judith@uwaterloo.ca,519-555-2343
1,Dominick Byrd,dominick@uwaterloo.ca,519-555-2325
2,Darrell Aucoin,darrell.aucoin@gmail.com,519-555-1424
3,Patrick Robertson,patrick@uwaterloo.ca,519-555-2312


## Subqueries

Subqueries are queries contained in queries. These subqueries are contained in '(', ')'

There are two types of subqueries:

1. __Non-Correlated Subqueries:__ Can be run independently of the larger query.

2. __Correlated Subqueries:__ Must be run concurrently with the outer query. They are dependent on the outer query.

### Non-Correlated Subqueries

__Non-Correlated Subquery:__ Any valid query within query that if executed by itself will produce a result (including empty set). These are enclosed in '(', ')' in __FROM__, __WHERE__, or __HAVING__ clauses.

__Example:__ What Stats Club execs are in charge of making posters?

```
SELECT e.name, e.position 
FROM exec AS e 
WHERE e.questid IN (SELECT poster FROM event);
```

In [13]:
pd.read_sql_query(
"SELECT e.name, e.position \
FROM exec AS e \
WHERE e.questid IN (SELECT poster FROM event)", con)

,name,position
0,Darrell Aucoin,President
1,Dominick Byrd,Events
2,Gilberto Cross,Events
3,Judith B. Gibbons,Events
4,Patrick Robertson,Events


### Correlated Subqueries

__Correlated  Subquery__ makes references to it's containing query, executing it for every candidate row referenced.
- Correlated subqueries can appear in the __SELECT__, __WHERE__, or __HAVING__ clauses.

__Example:__ What majors are the current Stats Club execs enrolled in?
- The member table has the information on majors but exec has the information on execs

```
SELECT name, position, 
(SELECT faculty FROM member AS m WHERE m.quest_id = e.questid) AS faulty, 
(SELECT major FROM member AS m WHERE m.quest_id = e.questid) AS major 
FROM exec AS e;
```

In [14]:
pd.read_sql_query(
"SELECT name, position, \
(SELECT faculty FROM member AS m WHERE m.quest_id = e.questid) AS faulty, \
(SELECT major FROM member AS m WHERE m.quest_id = e.questid) AS major \
FROM exec AS e", con)

,name,position,faulty,major
0,Darrell Aucoin,President,Math,Stats
1,Judith B. Gibbons,Events,Math,Act Sci
2,Lamar Roy,Finance,None,None
3,Gilberto Cross,Events,None,None
4,Melba Lane,President,None,None
5,Ruben Lamb,Technology,Math,Act Sci
6,Hannah Mason,SeniorAdvisor,None,None
7,Patrick Robertson,Events,Math,Stats
8,Dominick Byrd,Events,Math,Pure Math


### Correlated vs Non-Correlated

1. Correlated subquery is __dependent__ on outer query, non-correlated is __independent__.

2. Correlated subquery is executed __concurrently__ with outer query, non-correlated is executed before.

3. In general, for speed of execution:
$$\mbox{Correlated subquery}\prec\mbox{Non-Correlated subquery}\prec\mbox{Joins}$$
 

### Quiz: Subqueries

__Q:__ Where can a Non-Correlated subquery can be placed?

__A:__ in the __FROM__, __WHERE__, and __HAVING__ clauses

__Q:__ Where can a Correlated subquery can be placed?

__A:__ in the __SELECT__, __WHERE__, and __HAVING__ clauses

__Q:__ Using a non-correlated subquery, what are the names, locations, and descriptions of events that served pizza?
1. Break the problem into smaller pieces: What are the events that served pizza?
2. Only retrieve values from the table `event` that `event.name` matches those values

```
SELECT event 
FROM expenses 
WHERE expense = 'pizza';
```

In [15]:
pd.read_sql_query(
"SELECT event \
FROM expenses \
WHERE expense = 'pizza'", con)

,event
0,BOT
1,Intro to SQL
2,Prof Talk
3,Prof Talk 2


- Second, only retrieve values from the table `event` that `event.name` matches those values

```
SELECT name, location, description 
FROM event 
WHERE name IN (SELECT event FROM expenses WHERE expense = 'pizza');
```

In [16]:
pd.read_sql_query(
"SELECT name, location, description \
FROM event  \
WHERE name IN (SELECT event FROM expenses WHERE expense = 'pizza')", con)

,name,location,Description
0,BOT,C & D,Come and play games with your fellow stats Clu...
1,Intro to SQL,MC-3003,SQL is a relational database language and alon...
2,Prof Talk,M3-2134,Machine Learning and Data Mining: How We can C...
3,Prof Talk 2,None,None


Or alternatively, using a join:

```
SELECT e.name, e.location, e.description 
FROM (SELECT event FROM expenses WHERE expense = 'pizza') AS exp INNER JOIN event AS e 
  ON exp.event = e.name;
```

In [17]:
pd.read_sql_query(
"SELECT e.name, e.location, e.description \
FROM (SELECT event FROM expenses WHERE expense = 'pizza') AS exp INNER JOIN event AS e \
  ON exp.event = e.name", con)

,e.name,e.location,e.description
0,BOT,C & D,Come and play games with your fellow stats Clu...
1,Intro to SQL,MC-3003,SQL is a relational database language and alon...
2,Prof Talk,M3-2134,Machine Learning and Data Mining: How We can C...
3,Prof Talk 2,None,None


## Set Operations

Set operations create a combination of rows from 2 tables into one result set.
- The tables (or the projection of those tables) have to have the same number of columns and datatypes
    - if one column appears in a table but not in another, supply the value NULL for the missing column

1. __UNION__: For tables A and B, combines the rows for A and B into one result set.
2. __INTERSECT__: For tables A and B, returns the rows for A and B that they have in common.
3. __Difference (EXCEPT)__: For tables A and B, return only the rows in A that are not in common with B.

### UNION

__UNION operator:__ Addition of one result set to another result set with the same number of attributes and types.
```
SELECT ... FROM ... 
UNION [ALL]
SELECT ... FROM ...
```
- Just __UNION__ removes duplicates, while __UNION ALL__ keeps all rows from both result sets.
<center>
![alt text](Images/Union.png)
</center>

<center>
![alt text](Images/Union_all.png)
</center>

__Example:__ Suppose are you need to contact everyone involved in Stats Club: members and execs.  
What are the names, email addresses, and phone numbers (if you have them) of all Stats Club members and execs?

```
SELECT name, email, NULL AS phone 
FROM member 
UNION 
SELECT name, email, phone 
FROM exec;
```

In [18]:
pd.read_sql_query(
"SELECT name, email, NULL AS phone \
FROM member \
UNION \
SELECT name, email, phone \
FROM exec", con)

,name,email,phone
0,Ann W. McLaughlin,ann@uwaterloo.ca,None
1,Arnold J. Fuller,arnold@uwaterloo.ca,None
2,Austin K. Gilliard,austin@uwaterloo.ca,None
3,Billy L. Hunter,billy@uwaterloo.ca,None
4,Bobbie D. Mathews,bobbie@uwaterloo.ca,None
5,Daniel J. Moore,daniel@uwaterloo.ca,None
6,Darrell Aucoin,darrell.aucoin@gmail.com,None
7,Darrell Aucoin,darrell.aucoin@gmail.com,519-555-1424
8,Deborah D. Helfrich,deborah@uwaterloo.ca,None
9,Dominick Byrd,dominick@uwaterloo.ca,None


### Intersection (INTERSECT Operator)

__INTERSECT operator:__ Returns only tuples that are in common between two result sets. Result sets must be equal in number and type of attributes.
```
SELECT ... FROM ...
INTERSECT
SELECT ... FROM ...;
```
<center>
![alt text](Images/Intersect.png)
</center>

__Example:__ What Stats Club execs have also signed up as Stats Club members?

```
SELECT name, quest_id 
FROM member 
INTERSECT 
SELECT name, questid 
FROM exec;
```

In [19]:
pd.read_sql_query(
"SELECT name, quest_id \
FROM member \
INTERSECT \
SELECT name, questid \
FROM exec", con)

,name,quest_id
0,Darrell Aucoin,darrell
1,Dominick Byrd,dominick
2,Judith B. Gibbons,judith
3,Patrick Robertson,patrick
4,Ruben Lamb,ruben


### Difference (EXCEPT Operator)

__EXCEPT operator:__ Returns the first result set minus anything it has in common with the second result set.
```
SELECT ... FROM ...
EXCEPT [ALL]
SELECT ... FROM ...
```
- Just EXCEPT uses set theory version of minus. 

    - If B has a row in common with A then all rows matching that row is removed
<center>
![alt text](Images/Except.png)
</center>

- The optional ALL uses the bag semantics version.
    - If B has a row in common with A then only the number of common rows in B rows matching that row is removed
<center>
![alt text](Images/Except_all.png)
</center>

__Example:__ What Stats Club members are not execs?

```
SELECT name, quest_id 
FROM member 
EXCEPT 
SELECT name, questid 
FROM exec;
```

In [20]:
pd.read_sql_query(
"SELECT name, quest_id \
FROM member \
EXCEPT \
SELECT name, questid \
FROM exec", con)

,name,quest_id
0,Ann W. McLaughlin,ann
1,Arnold J. Fuller,arnold
2,Austin K. Gilliard,austin
3,Billy L. Hunter,billy
4,Bobbie D. Mathews,bobbie
5,Daniel J. Moore,daniel
6,Deborah D. Helfrich,deborah
7,Elaine S. Ott,elaine
8,Frances A. Miller,frances
9,Fred E. Finch,fred


## WITH Clause

__WITH clause:__ Makes a non-correlated subquery look like a table in the executed statement:

- Increases readability of the query as well as ensure that if it is used in several different places, it will only be executed once  

```
WITH subquery_name [(colname1, ...)] AS
	(SELECT ...),
	subquery_name2 [(colname1, ...)] AS
	(SELECT ...)
SELECT ... 
```

__Example:__ Suppose you are doing a report for MathSoc detailing Stats Club events, budget, and expenses. 

```
WITH cost (event, expenses) AS 
    (SELECT event, SUM(price) 
    FROM expenses 
    GROUP BY event) 
SELECT e.name, e.type, e.budget, cost.expenses 
FROM event AS e INNER JOIN cost ON e.name = cost.event;
```

In [21]:
pd.read_sql_query(
"WITH cost (event, expenses) AS \
    (SELECT event, SUM(price) \
    FROM expenses \
    GROUP BY event) \
SELECT e.name, e.type, e.budget, cost.expenses \
FROM event AS e INNER JOIN cost ON e.name = cost.event", con)

,name,type,budget,expenses
0,BOT,social,90.0,58.57
1,EOT,social,160.0,160.65
2,Intro to Hadoop,educational,90.0,96.81
3,Intro to SQL,educational,90.0,102.77
4,Intro to SQL: Advanced Queries,educational,100.0,62.63
5,Intro to SQL: Basic Queries,educational,100.0,62.63
6,Prof Talk,educational,90.0,82.87
7,Prof Talk 2,educational,90.0,77.21


### Quiz: WITH Clause (HARD)

__Q:__ MathSoc only provides a maximum cap on social events based on the formula below. What is the max cap for social expenses and is Stats Club over this limit?  
- Membership fee for Stats Club is 2 dollars

$$\begin{aligned}\mbox{Social expenses} & = & \max\left\{ 250,\left(\mbox{MathSoc Members}\right)\times\left(5+\left(\mbox{Member fee}\right)\right)\right\} \\
 & = & \max\left\{ 250,\left(\mbox{MathSoc Members}\right)\times7\right\} 
\end{aligned}$$  

Break the problem into smaller problems:
- What are the total expenses for social events?
- What is the max budget for social events?

Smaller Problems:
- What are the total expenses for social events?
    - What events are social events?

- What is the max cap for social events?
    - What is the result of the formula
    $$\left(\mbox{MathSoc Members}\right)\times7$$
        - What are the number of Stats Club members who are in the Math Faculty (aka MathSoc members)?
    - How do we find the max value between this number and 250?  
    
### Think about how to solve this for 5 mins

For total social expenses, we need to find the sum of expenses filtering out non-social expenses.  
__Part 1:__ What events are social?

```
SELECT name 
FROM event 
WHERE type = 'social';
```

In [22]:
pd.read_sql_query(
"SELECT name \
FROM event \
WHERE type = 'social'", con)

,name
0,BOT
1,EOT


__Part 2:__ What are the total social expenses?

```
SELECT SUM(price) 
FROM expenses 
WHERE event IN (SELECT name FROM event WHERE type = 'social');
```

In [23]:
pd.read_sql_query(
"SELECT SUM(price) \
FROM expenses \
WHERE event IN (SELECT name FROM event WHERE type = 'social')", con)

,SUM(price)
0,219.22


Or alternatively, we can join tables event and expenses and filter out non-social events:  

```
SELECT SUM(exp.price) 
FROM event AS e INNER JOIN expenses AS exp 
  ON e.name = exp.event 
WHERE e.type = 'social';
```

In [24]:
pd.read_sql_query(
"SELECT SUM(exp.price) \
FROM event AS e INNER JOIN expenses AS exp \
  ON e.name = exp.event \
WHERE e.type = 'social'", con)

,SUM(exp.price)
0,219.22


At this point we have one half of the problem solved, the final result would look a bit like:
```
WITH soc_cost (expense) AS 
    (SELECT SUM(exp.price) 
     FROM event AS e INNER JOIN expenses AS exp 
       ON e.name = exp.event 
     WHERE type = 'social'),
soc_budget (budget) AS
     (SELECT statement)
SELECT statement;
```

__Part 3:__ We need to calculate the formula
$$\left(\mbox{MathSoc Members}\right)\times7$$
- First, what is the number of MathSoc Members (aka number of members who are in the Math Faculty)?  

```
SELECT COUNT(*) 
FROM member 
WHERE faculty = 'Math';
```

In [25]:
pd.read_sql_query(
"SELECT COUNT(*) \
FROM member \
WHERE faculty = 'Math'", con)

,COUNT(*)
0,33


__Part 4:__ What is the result of the formula  
$$\left(\mbox{MathSoc Members}\right)\times7$$

```
SELECT COUNT(*) * 7 
FROM member 
WHERE faculty = 'Math';
```

In [26]:
pd.read_sql_query(
"SELECT COUNT(*) * 7 \
FROM member \
WHERE faculty = 'Math'", con)

,COUNT(*) * 7
0,231


__Part 5:__ What is the max of this number and 250?

```
SELECT MAX(soc_budget) 
FROM (SELECT COUNT(*) * 7 AS soc_budget 
      FROM member 
      WHERE faculty = 'Math' 
      UNION 
      SELECT 250);
```

In [27]:
pd.read_sql_query(
"SELECT MAX(soc_budget) \
FROM (SELECT COUNT(*) * 7 AS soc_budget \
FROM member \
WHERE faculty = 'Math' \
UNION \
SELECT 250)", con)

,MAX(soc_budget)
0,250


Joining all of this up together:

```
WITH soc_cost (expense) AS 
    (SELECT SUM(exp.price) 
     FROM event AS e INNER JOIN expenses AS exp 
       ON e.name = exp.event 
     WHERE e.type = 'social'),
soc_budget (budget) AS 
    (SELECT Max(soc_budget) 
     FROM (SELECT COUNT(*) * 7 AS soc_budget 
           FROM member 
           WHERE faculty = 'Math' 
           UNION 
           SELECT 250)
    )
SELECT expense, budget, budget - expense
FROM soc_cost CROSS JOIN soc_budget;
```

In [28]:
pd.read_sql_query(
"WITH soc_cost (expense) AS \
    (SELECT SUM(exp.price) \
     FROM event AS e INNER JOIN expenses AS exp \
       ON e.name = exp.event \
     WHERE e.type = 'social'), \
soc_budget (budget) AS \
    (SELECT Max(soc_budget) \
     FROM (SELECT COUNT(*) * 7 AS soc_budget \
           FROM member \
           WHERE faculty = 'Math' \
           UNION \
           SELECT 250) \
    ) \
SELECT expense, budget, budget - expense \
FROM soc_cost CROSS JOIN soc_budget", con)

,expense,budget,budget - expense
0,219.22,250,30.78


## CASE Expressions

__CASE expressions:__ Similar to a series of if else statements executed for every entry in a table. A new value is returned for every row in the table.

```
CASE [column]
	WHEN condition1 THEN result1
	WHEN condition2 THEN result2
	...
	WHEN condition_n THEN result_n
	[ELSE result]
END
```

- The result can be of any datatype or the result of a correlated or non-correlated subquery (if the result is a single value)

- CASE expressions are performed by themselves in the __SELECT__ clause or within a function or aggregate function

    - CASE expressions within aggregate functions allow us to do counts, sums, averages, etc. of particular occurrences

__Example:__ Suppose like before we are preparing a report for MathSoc, but now we want to give a warning if the event is over budget or not.

```
WITH cost (event, expenses) AS 
    (SELECT event, SUM(price) 
    FROM expenses 
    GROUP BY event) 
SELECT e.name, e.type, e.budget, cost.expenses, 
    CASE 
    WHEN e.budget - cost.expenses < 0 THEN 'Over budget' 
    ELSE NULL
END AS warning 
FROM event AS e INNER JOIN cost ON e.name = cost.event;
```

In [29]:
pd.read_sql_query(
"WITH cost (event, expenses) AS \
    (SELECT event, SUM(price) \
    FROM expenses \
    GROUP BY event) \
SELECT e.name, e.type, e.budget, cost.expenses, \
    CASE \
    WHEN e.budget - cost.expenses < 0 THEN 'Over budget' \
    ELSE NULL \
    END AS warning \
FROM event AS e INNER JOIN cost ON e.name = cost.event", con)

,name,type,budget,expenses,warning
0,BOT,social,90,58.57,None
1,EOT,social,160,160.65,Over budget
2,Intro to Hadoop,educational,90,96.81,Over budget
3,Intro to SQL,educational,90,102.77,Over budget
4,Intro to SQL: Advanced Queries,educational,100,62.63,None
5,Intro to SQL: Basic Queries,educational,100,62.63,None
6,Prof Talk,educational,90,82.87,None
7,Prof Talk 2,educational,90,77.21,None


### Quiz: CASE Expressions

__Q:__ Suppose we are interested in the healthiness of our food options at Stats Club events. A score of various foods was given below. What is the average 'healthiness' of Stats Club event's food options?
- The average of the score values for all of the foods at Stats Club events.

| Food | donuts | pop | fries | pizza | cookies | coffee | water | meals | veggie platter |
|------|--------|-----|-------|-------|---------|--------|-------|-------|----------------|
| Score | -2 | -2 | -2 | -1 | -1 | 0 | 2 | 2 | 3| 

```
SELECT AVG(CASE 
    WHEN expense IN ('donuts', 'pop', 'fries') THEN -2
    WHEN expense IN ('pizza', 'cookies') THEN -1
    WHEN expense IN ('coffee') THEN 0
    WHEN expense IN ('water', 'meals') THEN 2
    WHEN expense IN ('veggie platter') THEN 3
END) AS avg_score
FROM expenses;
```

In [30]:
pd.read_sql_query(
"SELECT AVG(CASE \
    WHEN expense IN ('donuts', 'pop', 'fries') THEN -2 \
    WHEN expense IN ('pizza', 'cookies') THEN -1 \
    WHEN expense IN ('coffee') THEN 0 \
    WHEN expense IN ('water', 'meals') THEN 2 \
    WHEN expense IN ('veggie platter') THEN 3 \
END) AS avg_score \
FROM expenses", con)

,avg_score
0,-0.75


## ORDER BY Clause

__ORDER BY Clause:__ Rearranges the rows of a result set according to a tuple of columns.

```
SELECT column_list 
FROM table_name 
ORDER BY column1, column2, .. columnN [ASC | DESC];
```
`ASC` is ascending (default)  

`DESC` is descending

__Example:__ What is the membership list ordered by name?

```
SELECT name 
FROM member 
ORDER BY name;
```

In [31]:
pd.read_sql_query(
"SELECT name \
FROM member \
ORDER BY name", con)

,name
0,Ann W. McLaughlin
1,Arnold J. Fuller
2,Austin K. Gilliard
3,Billy L. Hunter
4,Bobbie D. Mathews
5,Daniel J. Moore
6,Darrell Aucoin
7,Deborah D. Helfrich
8,Dominick Byrd
9,Elaine S. Ott


## LIMIT Clause

__LIMIT Clause:__ Restrict the result set to the first n of rows.

```
SELECT column1, column2, columnN 
FROM table_name
LIMIT no_of_rows;
```
- We can also offset where the restriction begins  

```
SELECT column1, column2, columnN 
FROM table_name
LIMIT no_of_rows OFFSET row_num;
```

__Example:__ What is the first 10 members ordered by name?

```
SELECT name 
FROM member 
ORDER BY name 
LIMIT 10;
```

In [32]:
pd.read_sql_query(
"SELECT name \
FROM member \
ORDER BY name \
LIMIT 10", con)

,name
0,Ann W. McLaughlin
1,Arnold J. Fuller
2,Austin K. Gilliard
3,Billy L. Hunter
4,Bobbie D. Mathews
5,Daniel J. Moore
6,Darrell Aucoin
7,Deborah D. Helfrich
8,Dominick Byrd
9,Elaine S. Ott


### Quiz: ORDER BY and LIMIT Clause

__Q:__ What are the top 10 highest priced items in expenses?  

```
SELECT expense, price
FROM expenses
ORDER BY price
LIMIT 10;
```

In [33]:
pd.read_sql_query(
"SELECT expense, price \
FROM expenses \
ORDER BY price \
LIMIT 10", con)

,expense,price
0,water,10.23
1,cookies,10.23
2,cookies,10.23
3,cookies,10.23
4,water,10.52
5,water,10.52
6,pop,13.23
7,pop,15.34
8,pop,15.65
9,pop,20.31


# Questions?

### Topics Not Covered
- How to create a database
    - Creating tables
    - Creating constraints
    - Indices
    - Views
    - How to insert, delete, alter, drop, etc. data in a table

### Next Stats Club Event: Intro to Hadoop 
#### Wednesday, March 25
#### 2:30-4:00pm
#### M3-3103

In [34]:
con.close()

# References

### Books:
_Learning SQL_, Second Edition, by Alan Beaulieu. Copyright 2009 O’Reilly Media, Inc., 978-0-596-52083-0.

### CS Database Courses:

__CS 338:__ Computer Applications in Business: Databases (CS Minors)

__CS 348:__ Introduction to Database Management (CS Majors)

### Coursera Courses:

__Introduction to Databases:__ by Jennifer Widom from Stanford  

__Getting and Cleaning Data:__ by Jeff Leek, PhD, Roger D. Peng, PhD, Brian Caffo, PhD from Johns Hopkins University

In [35]:
#sudo ipython nbconvert Intro_SQL_Presentation2.ipynb --to slides --post serve